Midi file player https://midiplayer.ehubsoft.net/

# Setup

In [ ]:
!curl -L https://cseweb.ucsd.edu/classes/sp25/cse253-a/data/student_files.tar.gz -o student_files.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1272M  100 1272M    0     0  26.3M      0  0:00:48  0:00:48 --:--:-- 19.8M


In [ ]:
# !rm student_files.tar.gz

In [ ]:
# Make sure to wait until the file fully finishes uploading
# !rm -rf student_files
!tar -xvzf student_files.tar.gz | tail -n 5  # Make sure there's no error message at the tail

student_files/task1_composer_classification/midis/1022.mid
student_files/task1_composer_classification/midis/147.mid
student_files/task1_composer_classification/midis/275.mid
student_files/task1_composer_classification/midis/765.mid
student_files/task1_composer_classification/midis/701.mid


In [ ]:
!pip install librosa | tail -n 1  # I don't want a super long output
!pip install miditoolkit | tail -n 1
!pip install xgboost | tail -n 1
!pip install lightgbm | tail -n 1

In [ ]:
# Probably more imports than are really necessary...
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from tqdm import tqdm
import librosa
import numpy as np
import miditoolkit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
import random

from mido import MidiFile
from sklearn.model_selection import train_test_split
from music21 import converter, chord, stream
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

# Getting data

In [ ]:
dataroot2 = "student_files/task2_next_sequence_prediction/"

### Old Solution

In [ ]:
# def features2(path):
#     full_path = dataroot2 + 'midis/' + path
#     midi_obj = miditoolkit.midi.parser.MidiFile(full_path)

#     notes = midi_obj.instruments[0].notes

    # # baseline (2 features)
    # num_notes = len(notes)
    # average_pitch = sum([note.pitch for note in notes]) / num_notes
    # average_duration = sum([note.end - note.start for note in notes]) / num_notes
    # features = [average_pitch, average_duration]

    # # notes durations and density (1 feature)
    # durations = [note.end - note.start for note in notes]
    # # std_dev_duration = np.std(durations)
    # mid = MidiFile(full_path)
    # total_time = mid.length  # in seconds
    # note_density = num_notes / total_time
    # features.extend([note_density])

    # # Volume (3 features)
    # velocities = np.array([note.velocity for note in notes])
    # # average_velocity = np.mean(velocities)
    # median_velocity = np.median(velocities)
    # std_dev_velocity = np.std(velocities)
    # velocity_range = np.max(velocities) - np.min(velocities)
    # features.extend([median_velocity, std_dev_velocity, velocity_range])

    # # Speed features (2 features)
    # note_onsets = np.array([note.start for note in notes])
    # onset_diffs = np.diff(note_onsets) # time difference between consecutive note onsets
    # average_onset_diff = np.mean(onset_diffs)
    # std_dev_onset_diff = np.std(onset_diffs)
    # # q1_onsets = np.percentile(onset_diffs, 25)
    # # q3_onsets = np.percentile(onset_diffs, 75)
    # # iqr_onsets = q3_onsets - q1_onsets
    # features.extend([average_onset_diff, std_dev_onset_diff])

    # # Get tempo changes (3 features)
    # tempo_changes = midi_obj.tempo_changes
    # tempo_values = []
    # tempo_times = []
    # average_tempo = 120 # Default tempo if no tempo changes
    # std_dev_tempo = 0
    # # num_tempo_changes = 0
    # if tempo_changes:
    #     # Sort tempo changes by time
    #     tempo_changes.sort(key=lambda x: x.time)
    #     tempo_values = np.array([tempo.tempo for tempo in tempo_changes])
    #     tempo_times = np.array([tempo.time for tempo in tempo_changes])

    #     average_tempo = np.mean(tempo_values)
    #     std_dev_tempo = np.std(tempo_values)
    #     # num_tempo_changes = len(tempo_values)
    # features.extend([average_tempo, std_dev_tempo])

    # # Register (4 features)
    # pitches = np.array([note.pitch for note in notes])
    # all_notes_range = np.max(pitches) - np.min(pitches) if pitches.size > 0 else 0
    # pitches_std_dev = np.std(pitches)
    # # Standard deviation of the highest and lowest 35% of notes (as a proxy for right hand and left hand melodies)
    # # Idk how accurate this is but let's try, I feel like if std dev is lower then it might be like Mozart and
    # # if it's higher it might be Chopin or smth
    # percentile = 35
    # sorted_pitches = np.sort(pitches)
    # num_highest_and_lowest_notes = int(np.ceil(num_notes * (percentile / 100)))
    # highest_pitches = sorted_pitches[-num_highest_and_lowest_notes:]
    # lowest_pitches = sorted_pitches[:num_highest_and_lowest_notes]
    # highest_notes_std_dev = np.std(highest_pitches)
    # lowest_notes_std_dev = np.std(lowest_pitches)
    # features.extend([all_notes_range, highest_notes_std_dev, lowest_notes_std_dev, pitches_std_dev])

    # # melodic pitch jumps (1 feature)
    # # notes_sorted = sorted(notes, key=lambda n: (n.start, -n.pitch))  # -pitch to get highest first
    # time_to_highest = {}  # get highest note at each time step
    # for note in notes:
    #     if note.start not in time_to_highest or note.pitch > time_to_highest[note.start]:
    #         time_to_highest[note.start] = note.pitch
    # melody_pitches = [time_to_highest[t] for t in sorted(time_to_highest)]
    # pitch_jumps = [abs(melody_pitches[i+1] - melody_pitches[i]) for i in range(len(melody_pitches) - 1)]
    # avg_jump = np.mean(pitch_jumps) if len(pitch_jumps) > 0 else 0
    # features.extend([avg_jump])

    # has_nan = np.isnan(np.array(features)).any()
    # if has_nan:
    #     print(path, "has NaN", features)

    # return features

### Current Solution

In [ ]:
# from collections import Counter

# def extract_chord(notes):
#     if not notes:
#         return ()

#     pitch_classes = [note.pitch % 12 for note in notes]
#     pc_counter = Counter(pitch_classes)

#     # Select the top 3 most common pitch classes (simple chord proxy)
#     top_3 = [pc for pc, _ in pc_counter.most_common(3)]
#     return tuple(sorted(top_3))  # e.g., (0, 4, 7) = C major

In [ ]:
def features2(path):
    full_path = dataroot2 + 'midis/' + path
    midi_obj = miditoolkit.midi.parser.MidiFile(full_path)

    notes = midi_obj.instruments[0].notes

    abs_features = []  # features to take the absolute value of between two midi files

    # tempo
    mean_tempo = 120.0  # default
    tempo_std = 0.0  # default
    tempos = midi_obj.tempo_changes
    if tempos:
        tempo_values = [t.tempo for t in tempos]
        mean_tempo = np.mean(tempo_values)
        tempo_std = np.std(tempo_values)
    abs_features.extend([mean_tempo, tempo_std])

    # time signature
    ts_numerator = 4  # default
    ts_denominator = 4
    tsigs = midi_obj.time_signature_changes
    if tsigs:
        ts_numerator = tsigs[0].numerator
        ts_denominator = tsigs[0].denominator
    abs_features.extend([ts_numerator, ts_denominator])

    # intervals
    time_to_highest = {}  # get highest note at each time step (melody)
    time_to_highest_vel = {}  # not for this feature, but will come in handy later
    for note in notes:
        if note.start not in time_to_highest or note.pitch > time_to_highest[note.start]:
            time_to_highest[note.start] = note.pitch
            time_to_highest_vel[note.start] = note.velocity
    melody_pitches = [time_to_highest[t] for t in sorted(time_to_highest)]
    melody_velocities = [time_to_highest_vel[t] for t in sorted(time_to_highest_vel)]  # not for this feature, but will come in handy later
    intervals = [abs(melody_pitches[i+1] - melody_pitches[i]) for i in range(len(melody_pitches) - 1)]
    avg_interval = 0  # default
    intervals_std = 0
    if len(intervals) > 0:
      avg_interval = np.mean(intervals)
      intervals_std = np.std(intervals)
    abs_features.extend([avg_interval, intervals_std])

    # ioi
    onsets = sorted([note.start for note in notes])
    ioi = np.diff(onsets)
    ioi_mean = 0  # default
    ioi_std = 0
    if len(ioi) > 0:
      ioi_mean = np.mean(ioi)
      ioi_std = np.std(ioi)
    abs_features.extend([ioi_mean, ioi_std])

    # average duration of left and right hand
    sorted_by_pitch = sorted(notes, key=lambda n: n.pitch)
    num_notes = len(sorted_by_pitch)
    # cutoff = int(np.ceil(num_notes * 0.35))  # use 35% as a proxy for left and right hand notes
    # left_notes = sorted_by_pitch[:cutoff]
    # right_notes = sorted_by_pitch[-cutoff:]
    # left_durations = [note.end - note.start for note in left_notes]
    # right_durations = [note.end - note.start for note in right_notes]
    # left_avg_duration = np.mean(left_durations) if left_durations else 0
    # right_avg_duration = np.mean(right_durations) if right_durations else 0
    # abs_features.extend([left_avg_duration, right_avg_duration])

    # note density
    # mid = MidiFile(full_path)
    # total_time = mid.length  # in seconds
    # note_density = num_notes / total_time
    # abs_features.extend([note_density])

    # pitch classes
    pitch_class_counts = [0] * 12
    for note in notes:
        pc = note.pitch % 12
        pitch_class_counts[pc] += 1
    pitch_class_distribution = [count / len(notes) for count in pitch_class_counts]
    abs_features.extend(pitch_class_distribution)

    # # ticks per beat
    # abs_features.extend([midi_obj.ticks_per_beat])

    # first and last pitches
    first_pitch = melody_pitches[0]
    last_pitch = melody_pitches[-1]

    # first and last tempos
    tempo_changes = midi_obj.tempo_changes
    first_tempo = tempo_changes[0].tempo if midi_obj.tempo_changes else 120.0 # Default if no tempo changes
    last_tempo = tempo_changes[-1].tempo if midi_obj.tempo_changes else 120.0

    # first and last velocities
    first_velocity = 0  # default
    last_velocity = 0
    if melody_velocities:
      first_velocity = melody_velocities[0]
      last_velocity = melody_velocities[-1]

    # first and last time signatures
    first_numerator = last_numerator = 4  # default
    first_denominator = last_denominator = 4
    if midi_obj.time_signature_changes:
      time_sigs = midi_obj.time_signature_changes
      time_sigs.sort(key=lambda x: x.time)
      first_ts = time_sigs[0]
      last_ts = time_sigs[-1]
      first_numerator = first_ts.numerator
      first_denominator = first_ts.denominator
      last_numerator = last_ts.numerator
      last_denominator = last_ts.denominator

    return (abs_features, first_pitch, last_pitch, first_tempo, last_tempo, first_velocity, last_velocity,
            first_numerator, first_denominator, last_numerator, last_denominator)

In [ ]:
def combineFeatures(features_1, features_2):
    # features has format (0 abs_features, 1 first_pitch, 2 last_pitch, 3 first_tempo,
    # 4 last_tempo, 5 first_velocity, 6 last_velocity, 7 first_numerator, 8 first_denominator,
    # 9 last_numerator, 10 last_denominator)
    combined_features = [abs(x - y) for x, y in zip(features_1[0], features_2[0])]

    # abs(last_pitch_1 - first_pitch_2)
    last_pitch_1 = features_1[2]
    first_pitch_2 = features_2[1]
    interval = abs(last_pitch_1 - first_pitch_2)
    combined_features.append(interval)

    # how many standard deviations is this interval off from the mean interval for both midi files?
    z_1 = (interval - features_1[0][4]) / features_1[0][5] if features_1[0][5] > 0 else 0
    z_2 = (interval - features_2[0][4]) / features_2[0][5] if features_2[0][5] > 0 else 0
    combined_features.extend([z_1, z_2])

    # abs(end_tempo_1 - start_tempo_2)
    end_tempo_1 = features_1[4]
    start_tempo_2 = features_2[3]
    combined_features.append(abs(start_tempo_2 - end_tempo_1))

    # velocity
    end_velocity_1 = features_1[6]
    start_velocity_2 = features_2[5]
    combined_features.append(abs(end_velocity_1 - start_velocity_2))

    # time signature
    end_numerator_1 = features_1[9]
    start_numerator_2 = features_2[7]
    combined_features.append(abs(end_numerator_1 - start_numerator_2))
    end_denominator_1 = features_1[10]
    start_denominator_2 = features_2[8]
    combined_features.append(abs(end_denominator_1 - start_denominator_2))

    return combined_features

In [ ]:
# We want to only get feature vectors once since they're costly, so save them in a dictionary
features = {}
midi_dir = dataroot2 + "midis/"
for f in os.listdir(midi_dir):
    features["midis/" + f] = features2(f)

In [ ]:
# Training data
train_path = dataroot2 + "train.json"

with open(train_path, 'r') as f:
    train_json = eval(f.read())

file_pairs_train_all = [k for k in train_json]
X_train_all = [combineFeatures(features[k1], features[k2]) for (k1, k2) in file_pairs_train_all]
y_train_all = [train_json[k] for k in train_json]

In [ ]:
X_train, X_val, y_train, y_val, file_pairs_train, file_pairs_val = train_test_split(
        X_train_all, y_train_all, file_pairs_train_all, test_size=0.2, random_state=42, shuffle=True
    )

In [ ]:
print(file_pairs_train_all[5])
print(X_train_all[5])
print(y_train_all[5])

('midis/277.mid', 'midis/4421.mid')
[np.float64(43.99984693347619), np.float64(0.0), 8, 12, np.float64(1.640289449112979), np.float64(3.203235798902703), np.float64(50.20626432391139), np.float64(25.060308506588974), 0.01318407960199005, 0.13706467661691543, 0.0017412935323383085, 0.11007462686567164, 0.12213930348258706, 0.0017412935323383085, 0.13507462686567165, 0.06629353233830845, 0.027363184079602004, 0.05348258706467661, 0.03955223880597014, 0.10348258706467661, 19, np.float64(2.516008896101268), np.float64(6.717672771012024), 43.99984693347619, 16, 8, 12]
False


In [ ]:
print(file_pairs_train[5])
print(X_train[5])
print(y_train[5])

('midis/1804.mid', 'midis/3392.mid')
[np.float64(0.0), np.float64(0.0), 0, 0, np.float64(4.16017316017316), np.float64(3.3261466034200797), np.float64(122.52964426877472), np.float64(115.26479133647302), 0.18856837606837606, 0.0, 0.05523504273504274, 0.09487179487179487, 0.044444444444444446, 0.3594017094017094, 0.03482905982905983, 0.33418803418803417, 0.044444444444444446, 0.08226495726495725, 0.060042735042735045, 0.08888888888888889, 7, np.float64(0.23652495839563298), np.float64(2.3770798118187906), 0.0, 0, 0, 0]
True


In [ ]:
print(file_pairs_val[5])
print(X_val[5])
print(y_val[5])

('midis/118.mid', 'midis/1058.mid')
[np.float64(0.0), np.float64(0.0), 0, 0, np.float64(4.933333333333332), np.float64(0.6249702817584453), np.float64(9.230769230769226), np.float64(0.3555563370856589), 0.13363139592647788, 0.07054148037754596, 0.05042225534028813, 0.02533532041728763, 0.0, 0.09860904123199205, 0.03676105315449578, 0.0, 0.021112767014406342, 0.0, 0.1174863387978142, 0.002483854942871337, 7, np.float64(-0.5777858063494004), np.float64(-0.9449825521403139), 0.0, 1, 0, 0]
True


In [ ]:
groundtruth_train_all = {k: train_json[k] for k in train_json}
groundtruth_train = {k: train_json[k] for k in file_pairs_train}
groundtruth_val = {k: train_json[k] for k in file_pairs_val}

In [ ]:
# Testing data
test_path = dataroot2 + "test.json"

d = eval(open(test_path, 'r').read())

file_pairs_test = [k for k in d]
X_test = [combineFeatures(features[k1], features[k2]) for (k1, k2) in file_pairs_test]

In [ ]:
print(file_pairs_test[5])
print(X_test[5])

('midis/5257.mid', 'midis/6086.mid')
[np.float64(0.0), np.float64(0.0), 0, 0, np.float64(4.502298850574713), np.float64(2.361707192462271), np.float64(32.76880901614378), np.float64(41.255875409854895), 0.0, 0.02035056446821154, 0.07635175282234107, 0.0008912655971479513, 0.11125965537730245, 0.0707070707070707, 0.023767082590612, 0.0, 0.08348187759952465, 0.044860368389780136, 0.0, 0.015745692216280444, 1, np.float64(-0.5511977325378299), np.float64(-1.0237986040928249), 0.0, 0, 0, 0]


# Eval Function

In [ ]:
def accuracy2(groundtruth, predictions):
    correct = 0
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        if predictions[k] == groundtruth[k]:
            correct += 1
    return correct / len(groundtruth)

# Model: Logistic Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
class logisticReg():
    def __init__(self):
        pass

    def train(self, X_train, y_train):
        model = LogisticRegression(max_iter=5000, random_state=42)
        model.fit(X_train, y_train)
        self.model = model

    def predict(self, file_pairs, features, outpath=None):
        predictions = {}
        for i, x in enumerate(features):
            k = file_pairs[i]
            pred = self.model.predict([x])
            predictions[k] = bool(pred[0])  # pred is [True] or [False]
            # print(k, pred)
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

In [ ]:
def runLogisticRegression():
    model = logisticReg()
    model.train(X_train, y_train)
    train_preds = model.predict(file_pairs_train, X_train)
    val_preds = model.predict(file_pairs_val, X_val)
    test_preds = model.predict(file_pairs_test, X_test, "predictions2.json")

    acc1_train = accuracy2(groundtruth_train, train_preds)
    acc1_val = accuracy2(groundtruth_val, val_preds)
    print("Task 1 training accuracy = " + str(acc1_train))
    print("Task 1 validation accuracy = " + str(acc1_val))

In [ ]:
!rm predictions2.json
runLogisticRegression()

Task 1 training accuracy = 0.9794663876536751
Task 1 validation accuracy = 0.9864016736401674


# Model: Decision Trees

# Model: XGBoost

In [ ]:
class XGBoost():
    def __init__(self):
        pass

    def predict(self, files, features, outpath=None):
        predictions = {}
        for i, x in enumerate(features):
            k = files[i]
            pred = self.model.predict([x])
            pred = self.label_encoder.inverse_transform(pred)
            predictions[k] = bool(pred[0])
            # print(k, pred[0])
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

    def train(self, X_train, y_train):
        # XGBoost wants outputs in [0 1] instead of [True False]
        # So we need to encode boolean labels to numerical values
        self.label_encoder = LabelEncoder()
        y_train_encoded = self.label_encoder.fit_transform(y_train)
        print(f"Classes found by LabelEncoder (should be 2): {self.label_encoder.classes_}")
        model = XGBClassifier(n_estimators=500, max_depth=15, learning_rate=0.1, objective='binary:logistic')
        model.fit(X_train, y_train_encoded)
        self.model = model

In [ ]:
def runXGBoost():
    model = XGBoost()
    model.train(X_train_all, y_train_all)
    train_preds = model.predict(file_pairs_train, X_train)
    val_preds = model.predict(file_pairs_val, X_val)
    test_preds = model.predict(file_pairs_test, X_test, "predictions2.json")

    acc1_train = accuracy2(groundtruth_train, train_preds)
    acc1_val = accuracy2(groundtruth_val, val_preds)
    print("Task 1 training accuracy = " + str(acc1_train))
    print("Task 1 validation accuracy = " + str(acc1_val))

In [ ]:
!rm predictions2.json
runXGBoost()

Classes found by LabelEncoder (should be 2): [False  True]
Task 1 training accuracy = 1.0
Task 1 validation accuracy = 1.0


# Model: Autogluon

# Model: MLP